<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/MNIST_Advanced_AI_Course_Project_1_Winter_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('Train: X=%s, y=%s' % (train_X.shape, train_y.shape))
print('Test: X=%s, y=%s' % (test_X.shape, test_y.shape))

In [ ]:
# ! rm -rf /content/drive

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
plt.hist(train_y, edgecolor='black', bins=np.arange(-0.5, 10))
plt.hist(test_y, edgecolor='black', bins=np.arange(-0.5, 10))
plt.subplots_adjust(top=1.4)
plt.legend(["training", "test"], loc ="top right")

plt.title('Distribution of Digits on Training and Test Dataset')
plt.xlabel('Digit')
plt.ylabel('Count')
plt.xticks(np.arange(0, 10))
plt.show()

In [ ]:
import shutil
import random

import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201

import cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(train_y[5])

In [ ]:
rows = 5
columns = 10
for digit in range(columns):
		selected = [index for index, value in enumerate(train_y) if value == digit][:rows]
		for index, value in enumerate(selected):
			k = index * columns + digit + 1
			plt.subplot(rows, columns, k)
			plt.imshow(train_X[value], cmap=plt.get_cmap('gray'))
			plt.axis('off')
			plt.subplots_adjust(wspace=0.1, hspace=0.01)
plt.show()

In [ ]:
train_X = train_X.reshape((-1, 28, 28, 1))
test_X = test_X.reshape((-1, 28, 28, 1))

In [ ]:
from tensorflow.keras.utils import to_categorical

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
# convert from integers to floats
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

# normalize to range 0-1
train_X = train_X / 255.0
test_X = test_X / 255.0

print(train_X.shape)
print(train_y.shape)

print(test_X.shape)
print(test_y.shape)

# Base Model


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape,MaxPooling2D

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from keras.callbacks import LearningRateScheduler

experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

history = model.fit(X, y, validation_split=0.2, epochs=30, callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/base-model/tensorboard")])

# Save and Load

## Save & Load History

In [ ]:
import pickle 

In [ ]:
with open('/content/drive/MyDrive/ai-gras-1/base-model/history.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
history = pickle.load(open('/content/drive/MyDrive/ai-gras-1/base-model/history.pkl', "rb"))

## Save & Load Model

In [ ]:
model.save('/content/drive/MyDrive/ai-gras-1/base-model')

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/ai-gras-1/base-model')

In [ ]:
print(model.summary())

In [ ]:
# plt.plot(history['accuracy'])
# plt.plot(history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='down right')
plt.show()

## Reading Tensorboard Log Files

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_accuracy')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

# Tunning

## Number of Convolution Layers Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  conv_layers_count = hp.Int("conv_layers_count", min_value=1, max_value=3, step=1)

  model = Sequential()
  if conv_layers_count == 1:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
  if conv_layers_count == 2:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
  if conv_layers_count == 3:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir",
    project_name="conv_layers_count"
)

In [ ]:
experiment_train_size = 10000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=3)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/conv_layers_count_dir

In [ ]:
!kill 1652

In [ ]:
tuner.search_space_summary(extended=True)

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback

In [ ]:
import tensorboard as tb


In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt

log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/5f715b4a8e21afc1a5205749e67df494/execution0/train/events.out.tfevents.1645893894.6401f1a17419.84.30.v2"
conv_3_train_x, conv_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/5f715b4a8e21afc1a5205749e67df494/execution0/validation/events.out.tfevents.1645893896.6401f1a17419.84.31.v2"
conv_3_validation_x, conv_3_validation_y = plot_tensorflow_log(log_file)


log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/61348a65887f0d4ce39edd0afe3e723e/execution0/train/events.out.tfevents.1645893860.6401f1a17419.84.27.v2"
conv_2_train_x, conv_2_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/61348a65887f0d4ce39edd0afe3e723e/execution0/validation/events.out.tfevents.1645893862.6401f1a17419.84.28.v2"
conv_2_validation_x, conv_2_validation_y = plot_tensorflow_log(log_file)


log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/786acfe532a6fd3021ab0737d768ea37/execution0/train/events.out.tfevents.1645893819.6401f1a17419.84.24.v2"
conv_1_train_x, conv_1_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/conv_layers_count_dir/callback/786acfe532a6fd3021ab0737d768ea37/execution0/validation/events.out.tfevents.1645893820.6401f1a17419.84.25.v2"
conv_1_validation_x, conv_1_validation_y = plot_tensorflow_log(log_file)

plt.plot(conv_3_train_x, conv_3_train_y, label='3 convolution layer - train')
plt.plot(conv_2_train_x, conv_2_train_y, label='2 convolution layer - train')
plt.plot(conv_1_train_x, conv_1_train_y, label='1 convolution layer - train')

plt.plot(conv_3_validation_x, conv_3_validation_y, '--', label='3 convolution layer - validation')
plt.plot(conv_2_validation_x, conv_2_validation_y, '--', label='2 convolution layer - validation')
plt.plot(conv_1_validation_x, conv_1_validation_y, '--', label='1 convolution layer - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

## Number of Hidden Layers Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  hidden_layers_count = hp.Int("hidden_layers_count", min_value=1, max_value=3, step=1)
  
  model = Sequential()

  model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  for i in range(hidden_layers_count):
      model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/dens_layers_count_dir",
    project_name="dens_layers_count"
)

In [ ]:
experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/dens_layers_count_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=3)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/dens_layers_count

In [ ]:
!kill 1652

In [ ]:
tuner.search_space_summary(extended=True)

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/dens_layers_count/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/310da671fde356bb178eb4a6ee01a7a6/execution0/train/events.out.tfevents.1645904681.6401f1a17419.84.55.v2"
dens_1_train_x, dens_1_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/310da671fde356bb178eb4a6ee01a7a6/execution0/validation/events.out.tfevents.1645904685.6401f1a17419.84.56.v2"
dens_1_validation_x, dens_1_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/87e54330d7895fbd79b1aa7c66e48fe4/execution0/train/events.out.tfevents.1645904598.6401f1a17419.84.52.v2"
dens_2_train_x, dens_2_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/87e54330d7895fbd79b1aa7c66e48fe4/execution0/validation/events.out.tfevents.1645904602.6401f1a17419.84.53.v2"
dens_2_validation_x, dens_2_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/90b0d45cc366dabad943cefe253ceb86/execution0/train/events.out.tfevents.1645904526.6401f1a17419.84.49.v2"
dens_3_train_x, dens_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dens_layers_count/callback/90b0d45cc366dabad943cefe253ceb86/execution0/validation/events.out.tfevents.1645904530.6401f1a17419.84.50.v2"
dens_3_validation_x, dens_3_validation_y = plot_tensorflow_log(log_file)

plt.plot(dens_3_train_x, dens_3_train_y, label='3 dens layer - train')
plt.plot(dens_2_train_x, dens_2_train_y, label='2 dens layer - train')
plt.plot(dens_1_train_x, dens_1_train_y, label='1 dens layer - train')

plt.plot(dens_3_validation_x, dens_3_validation_y, '--', label='3 dens layer - validation')
plt.plot(dens_2_validation_x, dens_2_validation_y, '--', label='2 dens layer - validation')
plt.plot(dens_1_validation_x, dens_1_validation_y, '--', label='1 dens layer - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

## Kernel Size Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  kernel_size = hp.Int("kernel_size", min_value=2, max_value=4, step=1)
  
  model = Sequential()

  model.add(Conv2D(16, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/kernel_size_dir",
    project_name="kernel_size"
)

In [ ]:
experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/kernel_size_dir

In [ ]:
! kill 5631

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/5b832d223d986ce101b771565403af91/execution0/train/events.out.tfevents.1645906151.6401f1a17419.84.73.v2"
kernel_4_train_x, kernel_4_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/5b832d223d986ce101b771565403af91/execution0/validation/events.out.tfevents.1645906155.6401f1a17419.84.74.v2"
kernel_4_validation_x, kernel_4_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/c0099ed2d158eac53288dfc66ac6fa62/execution0/train/events.out.tfevents.1645906068.6401f1a17419.84.70.v2"
kernel_3_train_x, kernel_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/c0099ed2d158eac53288dfc66ac6fa62/execution0/validation/events.out.tfevents.1645906072.6401f1a17419.84.71.v2"
kernel_3_validation_x, kernel_3_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/c463d84396ad1736a5f03ecfdbe09d62/execution0/train/events.out.tfevents.1645905992.6401f1a17419.84.67.v2"
kernel_2_train_x, kernel_2_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/kernel_size_dir/callback/c463d84396ad1736a5f03ecfdbe09d62/execution0/validation/events.out.tfevents.1645905996.6401f1a17419.84.68.v2"
kernel_2_validation_x, kernel_2_validation_y = plot_tensorflow_log(log_file)


plt.plot(kernel_4_train_x, kernel_4_train_y, label='(4, 4) kernel - train')
plt.plot(kernel_3_train_x, kernel_3_train_y, label='(3, 3) kernel - train')
plt.plot(kernel_2_train_x, kernel_2_train_y, label='(2, 2) kernel - train')

plt.plot(kernel_4_validation_x, kernel_4_validation_y, '--', label='(4, 4) kernel - validation')
plt.plot(kernel_3_validation_x, kernel_3_validation_y, '--', label='(3, 3) kernel - validation')
plt.plot(kernel_2_validation_x, kernel_2_validation_y, '--', label='(2, 2) kernel - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

## Stride Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  stride_size = hp.Int("stride_size", min_value=1, max_value=3, step=1)

  model = Sequential()

  kernel_size = 3
  model.add(Conv2D(16, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  # When stride is larger than 1, we cannot have 3 conv layers.
  # model.add(Conv2D(64, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  # model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/stride_size_dir",
    project_name="stride_size"
)

In [ ]:
experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/kernel_size_dir

In [ ]:
!kill 5631

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/stride_size_dir/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/992088f8a5eead3968655d87084349ee/execution0/train/events.out.tfevents.1645907110.6401f1a17419.84.82.v2"
stride_1_train_x, stride_1_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/992088f8a5eead3968655d87084349ee/execution0/validation/events.out.tfevents.1645907114.6401f1a17419.84.83.v2"
stride_1_validation_x, stride_1_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/30171d7697c15faa760ec54e3452a379/execution0/train/events.out.tfevents.1645906971.6401f1a17419.84.76.v2"
stride_3_train_x, stride_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/30171d7697c15faa760ec54e3452a379/execution0/validation/events.out.tfevents.1645906974.6401f1a17419.84.77.v2"
stride_3_validation_x, stride_3_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/30df4f775d475e507d2d249868357de1/execution0/train/events.out.tfevents.1645907027.6401f1a17419.84.79.v2"
stride_2_train_x, stride_2_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/stride_size_dir/callback/30df4f775d475e507d2d249868357de1/execution0/validation/events.out.tfevents.1645907030.6401f1a17419.84.80.v2"
stride_2_validation_x, stride_2_validation_y = plot_tensorflow_log(log_file)

plt.plot(stride_3_train_x, stride_3_train_y, label='(3, 3) stride - train')
plt.plot(stride_2_train_x, stride_2_train_y, label='(2, 2) stride - train')
plt.plot(stride_1_train_x, stride_1_train_y, label='(1, 1) stride - train')

plt.plot(stride_3_validation_x, stride_3_validation_y, '--', label='(3, 3) stride - validation')
plt.plot(stride_2_validation_x, stride_2_validation_y, '--', label='(2, 2) stride - validation')
plt.plot(stride_1_validation_x, stride_1_validation_y, '--', label='(1, 1) stride - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

## Pooling Size Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D


def build_model(hp):
  pooling_size = hp.Int("pooling_size", min_value=2, max_value=4, step=1)

  model = Sequential()

  stride_size = 1
  kernel_size = 3
  model.add(Conv2D(16, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/pooling_size_dir",
    project_name="pooling_size"
)

In [ ]:
experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/pooling_size_dir

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/cb57392469e65be1bf093b69a08c238c/execution0/train/events.out.tfevents.1645909670.8a122e665f20.71.25.v2"
pooling_4_train_x, pooling_4_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/cb57392469e65be1bf093b69a08c238c/execution0/validation/events.out.tfevents.1645909686.8a122e665f20.71.26.v2"
pooling_4_validation_x, pooling_4_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/ec8eae30827f57bb4d061789faf9a927/execution0/train/events.out.tfevents.1645908904.8a122e665f20.71.19.v2"
pooling_3_train_x, pooling_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/ec8eae30827f57bb4d061789faf9a927/execution0/validation/events.out.tfevents.1645908920.8a122e665f20.71.20.v2"
pooling_3_validation_x, pooling_3_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/e85e6847b52552a3beb6e2c34d936a37/execution0/train/events.out.tfevents.1645909287.8a122e665f20.71.22.v2"
pooling_2_train_x, pooling_2_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/pooling_size_dir/callback/e85e6847b52552a3beb6e2c34d936a37/execution0/validation/events.out.tfevents.1645909304.8a122e665f20.71.23.v2"
pooling_2_validation_x, pooling_2_validation_y = plot_tensorflow_log(log_file)

plt.plot(pooling_4_train_x, pooling_4_train_y, label='(4, 4) max pooling - train')
plt.plot(pooling_3_train_x, pooling_3_train_y, label='(3, 3) max pooling - train')
plt.plot(pooling_2_train_x, pooling_2_train_y, label='(2, 2) max pooling - train')

plt.plot(pooling_4_validation_x, pooling_4_validation_y, '--', label='(4, 4) max pooling - validation')
plt.plot(pooling_3_validation_x, pooling_3_validation_y, '--', label='(3, 3) max pooling - validation')
plt.plot(pooling_2_validation_x, pooling_2_validation_y, '--', label='(2, 2) max pooling - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import keras

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_hp

In [ ]:
best_models = tuner.get_best_models(num_models=1)

In [ ]:
import visualkeras
from PIL import ImageFont

font = ImageFont.truetype("/content/font/Roboto-Regular.ttf", 12)
visualkeras.layered_view(best_models[0], to_file='output.png', legend=True, font=font).show()

In [ ]:
best_models[0].get_config()

In [ ]:
best_models[0].summary()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
! ls /content/tf

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/conv_layers_count

## Dropout Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D


def build_model(hp):
  dropout = hp.Choice("dropout", [0.1, 0.3, 0.5])

  model = Sequential()

  stride_size = 1
  kernel_size = 3
  pooling_size = 4

  model.add(Conv2D(16, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(dropout))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/ai-gras-1/dropout_dir",
    project_name="dropout"
)

In [ ]:
experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/dropout_dir/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/pooling_size_dir

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/dropout_dir/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/19b43427bf22fc3f3d2c8cb8ffeddc24/execution0/train/events.out.tfevents.1645936062.422681e7c282.70.11.v2"
dropout_1_train_x, dropout_1_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/19b43427bf22fc3f3d2c8cb8ffeddc24/execution0/validation/events.out.tfevents.1645936077.422681e7c282.70.12.v2"
dropout_1_validation_x, dropout_1_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/dcf09ddea4ff8d89cb4591aa64ea6227/execution0/train/events.out.tfevents.1645936695.422681e7c282.70.17.v2"
dropout_3_train_x, dropout_3_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/dcf09ddea4ff8d89cb4591aa64ea6227/execution0/validation/events.out.tfevents.1645936710.422681e7c282.70.18.v2"
dropout_3_validation_x, dropout_3_validation_y = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/726c0d19a58adad10d9cdb9c404ac37f/execution0/train/events.out.tfevents.1645936372.422681e7c282.70.14.v2"
dropout_5_train_x, dropout_5_train_y = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/dropout_dir/callback/726c0d19a58adad10d9cdb9c404ac37f/execution0/validation/events.out.tfevents.1645936386.422681e7c282.70.15.v2"
dropout_5_validation_x, dropout_5_validation_y = plot_tensorflow_log(log_file)

plt.plot(dropout_1_train_x, dropout_1_train_y, label='0.1 dropout - train')
plt.plot(dropout_3_train_x, dropout_3_train_y, label='0.3 dropout - train')
plt.plot(dropout_5_train_x, dropout_5_train_y, label='0.5 dropout - train')

plt.plot(dropout_1_validation_x, dropout_1_validation_y, '--', label='0.1 dropout - validation')
plt.plot(dropout_3_validation_x, dropout_3_validation_y, '--', label='0.3 dropout - validation')
plt.plot(dropout_5_validation_x, dropout_5_validation_y, '--', label='0.5 dropout - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

## Others

In [ ]:
import keras

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_hp

In [ ]:
best_models = tuner.get_best_models(num_models=1)

In [ ]:
import visualkeras
from PIL import ImageFont

font = ImageFont.truetype("/content/font/Roboto-Regular.ttf", 12)
visualkeras.layered_view(best_models[0], to_file='output.png', legend=True, font=font).show()

In [ ]:
best_models[0].get_config()

In [ ]:
best_models[0].summary()

## Tensorboard Loading

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
! ls /content/tf

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/conv_layers_count

# Final Model

## Final Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape,MaxPooling2D

def base_model():
  model = Sequential()

  model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.1))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D


def final_model():
  model = Sequential()

  stride_size = 1
  kernel_size = 3
  pooling_size = 4

  model.add(Conv2D(16, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((pooling_size, pooling_size)))
  model.add(Conv2D(32, (kernel_size, kernel_size), strides=(stride_size, stride_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((pooling_size, pooling_size)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.1))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
from keras.callbacks import LearningRateScheduler

experiment_train_size = 60000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
model_base = base_model()
history_base = model_base.fit(X, y, validation_split=0.2, epochs=100, callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/base-model-100-2/tensorboard")])
model_base.save('/content/drive/MyDrive/ai-gras-1/base-model-100-2/save')

In [ ]:
model_final = final_model()
history_final = model_final.fit(X, y, validation_split=0.2, epochs=50, callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/final-model-100-5/tensorboard")])
model_final.save('/content/drive/MyDrive/ai-gras-1/final-model-100-5/save')

In [ ]:
! rm -rf /content/drive/MyDrive/ai-gras-1/final-model
! rm -rf /content/drive/MyDrive/ai-gras-1/base-model

In [ ]:
model_final.summary()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(18, 10))
ax[0].plot(history_final.history['loss'], color='b', label="Training loss")
ax[0].plot(history_final.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history_final.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history_final.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/base-model-100/tensorboard

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ai-gras-1/final-model-100-2/tensorboard

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/ai-gras-1/base-model-100/tensorboard/train/events.out.tfevents.1645922209.8a122e665f20.71.33.v2"
base_x_train, base_y_train = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/base-model-100/tensorboard/validation/events.out.tfevents.1645922251.8a122e665f20.71.34.v2"
base_x_validation, base_y_validation = plot_tensorflow_log(log_file)

log_file = "/content/drive/MyDrive/ai-gras-1/final-model-100/tensorboard/train/events.out.tfevents.1645926707.8a122e665f20.71.37.v2"
final_x_train, final_y_train = plot_tensorflow_log(log_file)
log_file = "/content/drive/MyDrive/ai-gras-1/final-model-100/tensorboard/validation/events.out.tfevents.1645926764.8a122e665f20.71.38.v2"
final_x_validation, final_y_validation = plot_tensorflow_log(log_file)

plt.plot(base_x_train, base_y_train, label='base model - train')
plt.plot(final_x_train, final_y_train, label='final model - train')

plt.plot(base_x_validation, base_y_validation, '--', label='base model - validation')
plt.plot(final_x_validation, final_y_validation, '--', label='final model - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
score, acc = base_model.evaluate(test_X, text_y)
print('Test score:', score)
print('Test accuracy:', acc)

## Final Model Test Accuracy

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/ai-gras-1/save')

In [ ]:
results = model.evaluate(test_X, test_y)

# System Spec

In [ ]:
! python --version

In [ ]:
! python -c 'import keras; print(keras.__version__)'

In [ ]:
! python -c 'import tensorflow as tf; print(tf.__version__)'

In [ ]:
! nvidia-smi -L

In [ ]:
! cat /proc/cpuinfo

# Installation

## Visual Keras

In [ ]:
! pip install git+https://github.com/paulgavrikov/visualkeras

## Keras Tuner

In [ ]:
! pip install keras-tuner --upgrade

## Fonts

In [ ]:
! apt install ttf-mscorefonts-installer

In [ ]:
! fc-cache -f

In [ ]:
! fc-match Arial

# Trash

In [ ]:
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.preprocessing.image import ImageDataGenerator
#Initalize Image Data Generator 
imageDataGenerator= ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,validation_split=0.2
)

# trainData = imageDataGenerator.flow(trainX, trainY, batch_size=64)
# valData = imageDataGenerator.flow(testX, testX, batch_size=64)
 # creating train data for our model
trainData = imageDataGenerator.flow(train_X,
                                    train_y,        
                                    batch_size=64,
                                    subset='training',
                                                    # shuffle=True,
                                                    
                                                   )
# creating validate data for our model
valData = imageDataGenerator.flow(train_X,
                                  train_y,
                                  batch_size=64,
                                  subset='validation'
                                                    # shuffle=True,
                                                    )
# # creating test data for our model
testData = imageDataGenerator.flow(train_X,
                                   train_y,         
                                   batch_size=1,    
                                   shuffle=True,
                                   seed=42)


In [ ]:
x_batch, y_batch = trainData.next()

plt.imshow(x_batch[:1].reshape(28, 28), cmap='gray')
plt.show()
# for j in range(64):


In [ ]:
train_X.shape

In [ ]:
new_X = np.empty(shape=(28, 28, 1))
for x_batch, y_batch in trainData:
 np.append(new_X, x_batch)

In [ ]:
X = trainData

In [ ]:
type(X)

In [ ]:
type(trainData)

In [ ]:
model = [0] *15

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape,MaxPooling2D

model[0] = Sequential()
model[0].add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model[0].add(MaxPooling2D((2, 2)))
model[0].add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model[0].add(MaxPooling2D((2, 2)))
model[0].add(Flatten())
model[0].add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
model[0].add(Dropout(0.3))
model[0].add(Dense(10, activation='softmax'))

# example output part of the model
# model.add(Flatten())
# model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(10, activation='softmax'))
model[0].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
try:
    os.mkdir("tmp")
except:
    print("Directory /tmp already exists")

checkpoint = tf.keras.callbacks.ModelCheckpoint("tmp/multi_class_weights_1", 
                                                monitor='val_loss', 
                                                verbose=1, 
                                                save_best_only=False,
                                                save_weights_only=True, 
                                                period=1)

In [ ]:
#train our model
from keras.callbacks import LearningRateScheduler

# history = model.fit(trainData, epochs = 20, validation_data =valData, steps_per_epoch=32, callbacks=[LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)], verbose=0)
# history = model.fit(trainData, epochs = 20, validation_data =valData, steps_per_epoch=32, callbacks=[checkpoint], verbose=0)
# dense_net_transfer_history = model.fit(trainData, validation_data=valData, epochs=40, steps_per_epoch=32, callbacks=[checkpoint])
history = model[0].fit(trainData, validation_data=valData, epochs=10, steps_per_epoch=32, callbacks=[checkpoint])

In [ ]:
acc = model[0].evaluate(testData, verbose=0)
# print('%.3f' % (acc * 100.0))
print(acc)

In [ ]:
#ino nemidoonam chiye
print("CNN: Epochs={0:d}, Train accuracy={1:.5f}, Validation accuracy={2:.5f}".format(epochs,max(history.history['acc']),max(history.history['val_acc']) ))


In [ ]:
from matplotlib import pyplot
# test harness for evaluating models on the cifar10 dataset
import sys
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	pyplot.show()
  # save plot to file
	# filename = sys.argv[0].split('/')[-1]
	# pyplot.savefig(filename + '_plot.png')
	# pyplot.close()

In [ ]:
summarize_diagnostics(history)